# PFT_GEM Tutorial: Tumor Growth Simulation with Tissue Compression

This notebook demonstrates how to use PFT_GEM (Posterior Fossa Tumor - Geometric Expansion Model) to simulate tumor-induced brain tissue displacement using a physically-motivated compression model.

## Key Physics

The simulation models tumor growth with these physical constraints:

1. **Fixed outer boundary**: The skull/outer boundary of the anatomy is fixed and cannot expand outward
2. **Tissue compression**: As the tumor grows, surrounding tissue is compressed (not just pushed)
3. **Gray matter**: Compresses uniformly in all directions (isotropic)
4. **White matter**: Resists stretching along fiber tracts (from DTI principal eigenvector)

## Contents

1. [Setup and Imports](#1.-Setup-and-Imports)
2. [Loading SUIT Template Data](#2.-Loading-SUIT-Template-Data)
3. [Loading DTI Reference Data](#3.-Loading-DTI-Reference-Data)
4. [Creating a Tumor Growth Simulation](#4.-Creating-a-Tumor-Growth-Simulation)
5. [Analyzing Tissue Compression](#5.-Analyzing-Tissue-Compression)
6. [Applying DTI-Based Anisotropic Constraints](#6.-Applying-DTI-Based-Anisotropic-Constraints)
7. [Saving Transforms and Results](#7.-Saving-Transforms-and-Results)
8. [Using Transforms with ANTsPy](#8.-Using-Transforms-with-ANTsPy)

## 1. Setup and Imports

In [ ]:
# Install PFT_GEM if not already installed
# !pip install -e ..

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
%matplotlib inline

# Import PFT_GEM modules
from pft_gem import (
    TumorGrowthSimulator,
    TumorGrowthParams,
    TissueProperties,
    simulate_tumor_growth,
    DisplacementField,
    BiophysicalConstraints,
)
from pft_gem.core.displacement import FieldMetadata
from pft_gem.core.constraints import DTIData, create_synthetic_dti
from pft_gem.io import SUITTemplateLoader, TemplateData
from pft_gem.visualization import (
    plot_displacement_magnitude,
    plot_vector_field,
    plot_jacobian,
    plot_slice_comparison,
    VisualizationConfig
)

# Check if ANTsPy is available
try:
    import ants
    HAS_ANTS = True
    print("ANTsPy is available!")
except ImportError:
    HAS_ANTS = False
    print("ANTsPy not available. Install with: pip install antspyx")

# Find the data directory
def find_data_dir():
    """Find the data directory regardless of current working directory."""
    possible_paths = [
        Path('../data'),
        Path('data'),
    ]
    
    try:
        import pft_gem
        pkg_path = Path(pft_gem.__file__).parent.parent / 'data'
        possible_paths.append(pkg_path)
    except (ImportError, AttributeError):
        pass
    
    for path in possible_paths:
        if path is not None and path.exists() and (path / 'suit_template').exists():
            return path.resolve()
    
    current = Path.cwd()
    for _ in range(5):
        candidate = current / 'data'
        if candidate.exists() and (candidate / 'suit_template').exists():
            return candidate.resolve()
        current = current.parent
    
    raise FileNotFoundError("Could not find data directory.")

DATA_DIR = find_data_dir()
SUIT_DIR = DATA_DIR / 'suit_template'
DTI_DIR = DATA_DIR / 'HCP1065_DTI'

print(f"\nPFT_GEM modules loaded successfully!")
print(f"Data directory: {DATA_DIR}")

## 2. Loading SUIT Template Data

The SUIT (Spatially Unbiased Infratentorial Template) provides high-resolution anatomical data for the cerebellum and brainstem.

In [ ]:
# Load SUIT template data
suit_loader = SUITTemplateLoader(SUIT_DIR)

if suit_loader.is_available():
    template_data = suit_loader.load_template()
    print("SUIT template loaded successfully!")
    print(f"Template shape: {template_data.template.shape}")
    print(f"Voxel size: {template_data.voxel_size} mm")
else:
    print("SUIT template not found. Creating synthetic template for demonstration.")
    template_data = SUITTemplateLoader.create_synthetic_template(
        shape=(128, 128, 64),
        voxel_size=(1.0, 1.0, 1.0)
    )

In [ ]:
# Visualize the SUIT template
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

shape = template_data.template.shape
cx, cy, cz = shape[0]//2, shape[1]//2, shape[2]//2

axes[0].imshow(template_data.template[cx, :, :].T, origin='lower', cmap='gray')
axes[0].set_title(f'Sagittal (x={cx})')
axes[1].imshow(template_data.template[:, cy, :].T, origin='lower', cmap='gray')
axes[1].set_title(f'Coronal (y={cy})')
axes[2].imshow(template_data.template[:, :, cz].T, origin='lower', cmap='gray')
axes[2].set_title(f'Axial (z={cz})')

plt.suptitle('SUIT Template', fontsize=14)
plt.tight_layout()
plt.show()

## 3. Loading DTI Reference Data

DTI data provides information about white matter fiber orientation. We use this to make white matter resist stretching along fiber tracts.

In [ ]:
import nibabel as nib

# Load DTI data
fa_path = DTI_DIR / 'FSL_HCP1065_FA_1mm.nii.gz'
v1_path = DTI_DIR / 'FSL_HCP1065_V1_1mm.nii.gz'

if fa_path.exists() and v1_path.exists():
    fa_img = nib.load(str(fa_path))
    v1_img = nib.load(str(v1_path))
    
    fa_full = fa_img.get_fdata()
    v1_full = v1_img.get_fdata()
    
    print("DTI data loaded successfully!")
    print(f"FA map shape: {fa_full.shape}")
    print(f"V1 (principal eigenvector) shape: {v1_full.shape}")
    print(f"FA range: [{fa_full.min():.3f}, {fa_full.max():.3f}]")
else:
    print("DTI data not found. Will create synthetic DTI data.")
    fa_full = None
    v1_full = None

## 4. Creating a Tumor Growth Simulation

Now we'll simulate tumor growth using the new physics-based model. The key difference from the old approach:

- **Old model**: Pushed tissue outward, ignoring boundary constraints
- **New model**: Compresses tissue between the tumor and fixed boundary

This means the Jacobian determinant will be < 1 around the tumor (compression), not > 1 (expansion).

In [ ]:
# Get template properties
grid_shape = template_data.template.shape
voxel_size = template_data.voxel_size
affine = template_data.affine if template_data.affine is not None else np.eye(4)

# Define tumor location - in the cerebellar hemisphere
# Convert to world coordinates (mm)
tumor_center_voxel = (
    grid_shape[0] * 0.35,
    grid_shape[1] * 0.55,
    grid_shape[2] * 0.45
)

# Convert to world coordinates
tumor_center_world = (
    tumor_center_voxel[0] * voxel_size[0] + affine[0, 3],
    tumor_center_voxel[1] * voxel_size[1] + affine[1, 3],
    tumor_center_voxel[2] * voxel_size[2] + affine[2, 3]
)

tumor_radius = 12.0  # 12mm radius (24mm diameter tumor)

print(f"Grid shape: {grid_shape}")
print(f"Voxel size: {voxel_size} mm")
print(f"Tumor center (voxels): ({tumor_center_voxel[0]:.1f}, {tumor_center_voxel[1]:.1f}, {tumor_center_voxel[2]:.1f})")
print(f"Tumor center (mm): ({tumor_center_world[0]:.1f}, {tumor_center_world[1]:.1f}, {tumor_center_world[2]:.1f})")
print(f"Tumor radius: {tumor_radius} mm")

In [ ]:
# Create tumor growth parameters
tumor_params = TumorGrowthParams(
    center=tumor_center_world,
    initial_radius=0.0,      # No pre-existing tumor
    final_radius=tumor_radius,
    shape="spherical"
)

# Define tissue mechanical properties
tissue_props = TissueProperties(
    gray_matter_compressibility=0.8,    # GM compresses easily
    white_matter_compressibility=0.6,   # WM is stiffer
    white_matter_anisotropy=0.7,        # Strong fiber resistance
    csf_compressibility=0.95,           # CSF compresses very easily
    boundary_stiffness=1.0              # Fixed boundary
)

print(f"Tumor growth volume: {tumor_params.growth_volume:.1f} mm^3")
print(f"\nTissue properties:")
print(f"  Gray matter compressibility: {tissue_props.gray_matter_compressibility}")
print(f"  White matter compressibility: {tissue_props.white_matter_compressibility}")
print(f"  White matter anisotropy: {tissue_props.white_matter_anisotropy}")

In [ ]:
# Create the simulator
boundary_mask = template_data.mask if template_data.mask is not None else template_data.template > 0

simulator = TumorGrowthSimulator(
    template_image=template_data.template,
    boundary_mask=boundary_mask,
    voxel_size=voxel_size,
    affine=affine
)

print("Tumor growth simulator created!")
print(f"Using ANTsPy: {HAS_ANTS}")

In [ ]:
# Run the simulation
output = simulator.simulate_growth(
    tumor_params=tumor_params,
    tissue_props=tissue_props,
    use_ants=HAS_ANTS
)

print("Simulation complete!")
print(f"\nDisplacement field shape: {output.displacement_field.shape}")
print(f"Jacobian determinant range: [{output.jacobian_determinant.min():.3f}, {output.jacobian_determinant.max():.3f}]")

# Print metadata
print(f"\nDisplacement statistics:")
print(f"  Max displacement: {output.metadata['displacement_stats']['max_mm']:.2f} mm")
print(f"  Mean displacement: {output.metadata['displacement_stats']['mean_mm']:.2f} mm")

print(f"\nCompression statistics:")
print(f"  Mean Jacobian: {output.metadata['compression_stats']['mean_jacobian']:.3f}")
print(f"  Voxels compressed (J<0.99): {output.metadata['compression_stats']['voxels_compressed']}")

In [ ]:
# Visualize the results
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

slice_x = int(tumor_center_voxel[0])
slice_y = int(tumor_center_voxel[1])
slice_z = int(tumor_center_voxel[2])

# Original template
axes[0, 0].imshow(output.original_image[slice_x, :, :].T, origin='lower', cmap='gray')
axes[0, 0].set_title('Original - Sagittal')
axes[0, 1].imshow(output.original_image[:, slice_y, :].T, origin='lower', cmap='gray')
axes[0, 1].set_title('Original - Coronal')
axes[0, 2].imshow(output.original_image[:, :, slice_z].T, origin='lower', cmap='gray')
axes[0, 2].set_title('Original - Axial')

# Displaced template with tumor
axes[1, 0].imshow(output.displaced_image[slice_x, :, :].T, origin='lower', cmap='gray')
axes[1, 0].contour(output.tumor_mask[slice_x, :, :].T, levels=[0.5], colors='red', linewidths=2)
axes[1, 0].set_title('Displaced - Sagittal')

axes[1, 1].imshow(output.displaced_image[:, slice_y, :].T, origin='lower', cmap='gray')
axes[1, 1].contour(output.tumor_mask[:, slice_y, :].T, levels=[0.5], colors='red', linewidths=2)
axes[1, 1].set_title('Displaced - Coronal')

axes[1, 2].imshow(output.displaced_image[:, :, slice_z].T, origin='lower', cmap='gray')
axes[1, 2].contour(output.tumor_mask[:, :, slice_z].T, levels=[0.5], colors='red', linewidths=2)
axes[1, 2].set_title('Displaced - Axial')

plt.suptitle('Original vs Displaced Template (tumor boundary in red)', fontsize=14)
plt.tight_layout()
plt.show()

## 5. Analyzing Tissue Compression

The Jacobian determinant tells us about local volume change:
- **J < 1**: Compression (tissue is squeezed)
- **J = 1**: No volume change
- **J > 1**: Expansion

With fixed boundaries, we expect J < 1 around the tumor (compression), not expansion.

In [ ]:
# Visualize Jacobian determinant (compression map)
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

jacobian = output.jacobian_determinant

# Use symmetric colormap centered at 1
vmin = min(jacobian.min(), 2 - jacobian.max())
vmax = max(jacobian.max(), 2 - jacobian.min())
vmin = max(vmin, 0.5)
vmax = min(vmax, 1.5)

im0 = axes[0].imshow(jacobian[slice_x, :, :].T, origin='lower', cmap='RdBu_r', vmin=vmin, vmax=vmax)
axes[0].contour(output.tumor_mask[slice_x, :, :].T, levels=[0.5], colors='black', linewidths=2)
axes[0].set_title('Jacobian - Sagittal')

im1 = axes[1].imshow(jacobian[:, slice_y, :].T, origin='lower', cmap='RdBu_r', vmin=vmin, vmax=vmax)
axes[1].contour(output.tumor_mask[:, slice_y, :].T, levels=[0.5], colors='black', linewidths=2)
axes[1].set_title('Jacobian - Coronal')

im2 = axes[2].imshow(jacobian[:, :, slice_z].T, origin='lower', cmap='RdBu_r', vmin=vmin, vmax=vmax)
axes[2].contour(output.tumor_mask[:, :, slice_z].T, levels=[0.5], colors='black', linewidths=2)
axes[2].set_title('Jacobian - Axial')

fig.colorbar(im2, ax=axes, label='Jacobian (< 1 = compression)', shrink=0.8)
plt.suptitle('Jacobian Determinant (Local Volume Change)\nBlue = Compression, Red = Expansion', fontsize=14)
plt.tight_layout()
plt.show()

# Statistics
tissue_region = (boundary_mask > 0) & (output.tumor_mask == 0)
print(f"\nJacobian statistics in tissue region:")
print(f"  Minimum (max compression): {jacobian[tissue_region].min():.3f}")
print(f"  Maximum: {jacobian[tissue_region].max():.3f}")
print(f"  Mean: {jacobian[tissue_region].mean():.3f}")
print(f"  % voxels with J < 0.95 (>5% compression): {100*np.sum((jacobian < 0.95) & tissue_region)/np.sum(tissue_region):.1f}%")

In [ ]:
# Visualize displacement magnitude
displacement_mag = np.linalg.norm(output.displacement_field, axis=-1)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

im0 = axes[0].imshow(displacement_mag[slice_x, :, :].T, origin='lower', cmap='hot')
axes[0].contour(output.tumor_mask[slice_x, :, :].T, levels=[0.5], colors='cyan', linewidths=2)
axes[0].set_title('Displacement Magnitude - Sagittal')

im1 = axes[1].imshow(displacement_mag[:, slice_y, :].T, origin='lower', cmap='hot')
axes[1].contour(output.tumor_mask[:, slice_y, :].T, levels=[0.5], colors='cyan', linewidths=2)
axes[1].set_title('Displacement Magnitude - Coronal')

im2 = axes[2].imshow(displacement_mag[:, :, slice_z].T, origin='lower', cmap='hot')
axes[2].contour(output.tumor_mask[:, :, slice_z].T, levels=[0.5], colors='cyan', linewidths=2)
axes[2].set_title('Displacement Magnitude - Axial')

fig.colorbar(im2, ax=axes, label='Displacement (mm)', shrink=0.8)
plt.suptitle('Displacement Field Magnitude (tumor boundary in cyan)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Plot displacement as a function of distance from tumor
from scipy import ndimage

# Compute distance from tumor center
x = np.arange(grid_shape[0]) * voxel_size[0] + affine[0, 3]
y = np.arange(grid_shape[1]) * voxel_size[1] + affine[1, 3]
z = np.arange(grid_shape[2]) * voxel_size[2] + affine[2, 3]
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

dist_from_tumor_center = np.sqrt(
    (X - tumor_center_world[0])**2 +
    (Y - tumor_center_world[1])**2 +
    (Z - tumor_center_world[2])**2
)

# Sample displacement vs distance
distances = []
displacements = []
jacobians = []

for d in np.linspace(0, 50, 100):
    mask = (dist_from_tumor_center > d - 1) & (dist_from_tumor_center < d + 1) & (boundary_mask > 0)
    if np.any(mask):
        distances.append(d)
        displacements.append(displacement_mag[mask].mean())
        jacobians.append(jacobian[mask].mean())

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].plot(distances, displacements, 'b-', linewidth=2)
axes[0].axvline(x=tumor_radius, color='r', linestyle='--', label='Tumor boundary')
axes[0].set_xlabel('Distance from tumor center (mm)')
axes[0].set_ylabel('Mean displacement magnitude (mm)')
axes[0].set_title('Displacement Profile')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(distances, jacobians, 'g-', linewidth=2)
axes[1].axvline(x=tumor_radius, color='r', linestyle='--', label='Tumor boundary')
axes[1].axhline(y=1.0, color='gray', linestyle=':', label='No change')
axes[1].set_xlabel('Distance from tumor center (mm)')
axes[1].set_ylabel('Mean Jacobian determinant')
axes[1].set_title('Compression Profile')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.suptitle('Radial Profiles from Tumor Center', fontsize=14)
plt.tight_layout()
plt.show()

## 6. Applying DTI-Based Anisotropic Constraints

White matter fiber tracts resist stretching along their principal direction. We use DTI data (FA and V1) to model this anisotropy.

In [ ]:
# Create synthetic DTI data for demonstration
# (In practice, you would use real DTI data registered to SUIT space)

dti_data = create_synthetic_dti(
    shape=grid_shape,
    tumor_center=(int(tumor_center_voxel[0]), int(tumor_center_voxel[1]), int(tumor_center_voxel[2])),
    tumor_radius=tumor_radius
)

print(f"Synthetic DTI data created:")
print(f"  FA range: [{dti_data.fa.min():.3f}, {dti_data.fa.max():.3f}]")
print(f"  MD range: [{dti_data.md.min()*1000:.2f}, {dti_data.md.max()*1000:.2f}] x 10^-3 mm^2/s")
print(f"  V1 shape: {dti_data.v1.shape}")

In [ ]:
# Create tissue mask from DTI (GM=1, WM=2, CSF=3)
constraints = BiophysicalConstraints(dti_data)

wm_mask = constraints.get_tissue_mask('white_matter')
gm_mask = constraints.get_tissue_mask('gray_matter')

tissue_mask = np.zeros(grid_shape, dtype=np.uint8)
tissue_mask[gm_mask > 0] = 1  # Gray matter
tissue_mask[wm_mask > 0] = 2  # White matter

print(f"Tissue classification:")
print(f"  White matter voxels: {np.sum(wm_mask)}")
print(f"  Gray matter voxels: {np.sum(gm_mask)}")

In [ ]:
# Visualize DTI data
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

im0 = axes[0].imshow(dti_data.fa[:, :, slice_z].T, origin='lower', cmap='viridis', vmin=0, vmax=1)
axes[0].contour(output.tumor_mask[:, :, slice_z].T, levels=[0.5], colors='red', linewidths=2)
axes[0].set_title('Fractional Anisotropy (FA)')
plt.colorbar(im0, ax=axes[0])

# V1 as RGB (direction encoded)
v1_rgb = np.abs(dti_data.v1[:, :, slice_z, :])
v1_rgb = v1_rgb / (v1_rgb.max() + 1e-10)
axes[1].imshow(np.transpose(v1_rgb, (1, 0, 2)), origin='lower')
axes[1].contour(output.tumor_mask[:, :, slice_z].T, levels=[0.5], colors='white', linewidths=2)
axes[1].set_title('V1 Direction (RGB = XYZ)')

im2 = axes[2].imshow(tissue_mask[:, :, slice_z].T, origin='lower', cmap='Set1', vmin=0, vmax=3)
axes[2].contour(output.tumor_mask[:, :, slice_z].T, levels=[0.5], colors='black', linewidths=2)
axes[2].set_title('Tissue Classification\n(1=GM, 2=WM)')

plt.tight_layout()
plt.show()

In [ ]:
# Re-run simulation with DTI constraints
simulator_dti = TumorGrowthSimulator(
    template_image=template_data.template,
    boundary_mask=boundary_mask,
    voxel_size=voxel_size,
    affine=affine,
    tissue_mask=tissue_mask,
    dti_v1=dti_data.v1,
    dti_fa=dti_data.fa
)

output_dti = simulator_dti.simulate_growth(
    tumor_params=tumor_params,
    tissue_props=tissue_props,
    use_ants=HAS_ANTS
)

print("Simulation with DTI constraints complete!")
print(f"\nWithout DTI:")
print(f"  Max displacement: {output.metadata['displacement_stats']['max_mm']:.2f} mm")
print(f"\nWith DTI constraints:")
print(f"  Max displacement: {output_dti.metadata['displacement_stats']['max_mm']:.2f} mm")

In [ ]:
# Compare displacement with and without DTI constraints
disp_mag_dti = np.linalg.norm(output_dti.displacement_field, axis=-1)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

vmax = max(displacement_mag.max(), disp_mag_dti.max())

im0 = axes[0].imshow(displacement_mag[:, :, slice_z].T, origin='lower', cmap='hot', vmax=vmax)
axes[0].contour(output.tumor_mask[:, :, slice_z].T, levels=[0.5], colors='cyan', linewidths=2)
axes[0].set_title('Without DTI Constraints')
plt.colorbar(im0, ax=axes[0], label='mm')

im1 = axes[1].imshow(disp_mag_dti[:, :, slice_z].T, origin='lower', cmap='hot', vmax=vmax)
axes[1].contour(output_dti.tumor_mask[:, :, slice_z].T, levels=[0.5], colors='cyan', linewidths=2)
axes[1].set_title('With DTI Constraints')
plt.colorbar(im1, ax=axes[1], label='mm')

# Difference
diff = disp_mag_dti - displacement_mag
vmax_diff = np.abs(diff).max()
im2 = axes[2].imshow(diff[:, :, slice_z].T, origin='lower', cmap='RdBu_r', vmin=-vmax_diff, vmax=vmax_diff)
axes[2].contour(output.tumor_mask[:, :, slice_z].T, levels=[0.5], colors='black', linewidths=2)
axes[2].set_title('Difference (DTI - baseline)')
plt.colorbar(im2, ax=axes[2], label='mm')

plt.suptitle('Effect of DTI Anisotropy on Displacement', fontsize=14)
plt.tight_layout()
plt.show()

## 7. Saving Transforms and Results

The simulation saves multiple output files:
- **Displaced image**: The warped template
- **Forward warp**: Displacement field that maps original to displaced
- **Inverse warp**: Displacement field that maps displaced back to original
- **Jacobian**: Local volume change map
- **Metadata**: Simulation parameters and statistics

In [ ]:
# Save all outputs
output_dir = DATA_DIR.parent / 'output'
output_dir.mkdir(exist_ok=True)

saved_paths = simulator_dti.save_transforms(
    output_dti,
    output_dir,
    prefix="tumor_growth_dti"
)

print("Saved files:")
for name, path in saved_paths.items():
    print(f"  {name}: {path.name}")

In [ ]:
# View metadata
import json

print("Simulation Metadata:")
print(json.dumps(output_dti.metadata, indent=2, default=str))

## 8. Using Transforms with ANTsPy

The saved warp files are compatible with ANTsPy and other neuroimaging tools.

In [ ]:
# Demonstrate using the warp with ANTsPy (if available)
if HAS_ANTS:
    print("Demonstrating ANTsPy transform application...")
    
    # Load the template as ANTs image
    ants_template = ants.from_numpy(
        template_data.template.astype(np.float32),
        spacing=voxel_size,
        origin=(affine[0, 3], affine[1, 3], affine[2, 3])
    )
    
    # Load the warp field
    warp_path = str(saved_paths['warp'])
    print(f"\nWarp file: {warp_path}")
    
    # The warp can be applied using antsApplyTransforms
    print("\nTo apply the warp to another image:")
    print(f"  antsApplyTransforms -d 3 -i input.nii.gz -r reference.nii.gz \\")
    print(f"    -t {warp_path} -o output.nii.gz")
    
    print(f"\nTo apply the inverse warp:")
    print(f"  antsApplyTransforms -d 3 -i input.nii.gz -r reference.nii.gz \\")
    print(f"    -t {saved_paths['warp_inverse']} -o output.nii.gz")
else:
    print("ANTsPy not available. The warp files can still be used with command-line ANTs tools.")
    print("\nTo apply the warp:")
    print(f"  antsApplyTransforms -d 3 -i input.nii.gz -r reference.nii.gz \\")
    print(f"    -t {saved_paths['warp']} -o output.nii.gz")

In [ ]:
# Apply warp to the atlas to see how regions are displaced
if template_data.atlas is not None:
    from scipy import ndimage as ndi
    
    # Warp the atlas using nearest neighbor (to preserve labels)
    coords = np.indices(grid_shape).astype(np.float64)
    for i in range(3):
        coords[i] += output_dti.displacement_field[..., i] / voxel_size[i]
    
    warped_atlas = ndi.map_coordinates(
        template_data.atlas.astype(np.float32),
        coords,
        order=0,  # Nearest neighbor
        mode='constant',
        cval=0
    )
    
    # Visualize
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(template_data.atlas[:, :, slice_z].T, origin='lower', cmap='nipy_spectral')
    axes[0].set_title('Original Atlas')
    
    axes[1].imshow(warped_atlas[:, :, slice_z].T, origin='lower', cmap='nipy_spectral')
    axes[1].contour(output_dti.tumor_mask[:, :, slice_z].T, levels=[0.5], colors='white', linewidths=2)
    axes[1].set_title('Warped Atlas (with tumor)')
    
    # Show affected regions
    diff_atlas = np.abs(warped_atlas - template_data.atlas)
    axes[2].imshow(diff_atlas[:, :, slice_z].T > 0, origin='lower', cmap='Reds')
    axes[2].contour(output_dti.tumor_mask[:, :, slice_z].T, levels=[0.5], colors='cyan', linewidths=2)
    axes[2].set_title('Displaced Regions')
    
    plt.suptitle('Atlas Deformation by Tumor Growth', fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("Atlas not available for this template.")

## Summary

This tutorial demonstrated the new physics-based tumor growth simulation:

### Key Features

1. **Fixed boundary constraint**: The outer boundary (skull) is fixed, so tissue must compress
2. **Compression model**: Jacobian < 1 around tumor indicates tissue compression
3. **DTI anisotropy**: White matter resists stretching along fiber direction
4. **ANTsPy compatibility**: Transforms are saved in standard NIfTI format

### Output Files

- `*_displaced.nii.gz`: Warped template showing tumor effect
- `*_warp.nii.gz`: Forward displacement field (original → displaced)
- `*_warp_inverse.nii.gz`: Inverse displacement field (displaced → original)
- `*_jacobian.nii.gz`: Local volume change map
- `*_tumor_mask.nii.gz`: Binary tumor region
- `*_metadata.json`: Simulation parameters

### Physical Interpretation

- **Jacobian < 1**: Tissue is compressed (squeezed by tumor)
- **Jacobian = 1**: No local volume change
- **Displacement → 0 at boundary**: Fixed boundary condition satisfied

This model provides a more physically realistic simulation than the original radial expansion model, which did not account for the fixed skull constraint.